In [31]:
from __future__ import print_function
import torch
import torchvision

from torch import nn
from torch.nn import functional as F
from torchvision import datasets, transforms

class LeNet_2(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, 1, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, 1, padding=1)
    self.conv3 = nn.Conv2d(32, 64, 3, 1, padding=1)
    self.fc1 = nn.Linear(4*4*64, 10)
    self.dropout1 = nn.Dropout(0.5)
    
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.conv3(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.dropout1(x)
    x = x.view(-1, 4*4*64)
    x = F.relu(self.fc1(x))
    return x

device = torch.device("cuda:0")
model = LeNet_2().to(device)

transformer = transforms.Compose([transforms.Resize((32, 32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                 ])

train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transformer)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transformer)

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=1, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0)

epochs = 12
running_loss_history = []
running_correct_history = []

for e in range(epochs):
  
  running_loss = 0.0
  running_correct = 0.0
  validation_running_loss = 0.0
  validation_running_correct = 0.0
  
  for inputs, labels in train_loader:    
     
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _, preds = torch.max(outputs, 1)
    
    running_correct += torch.sum(preds == labels.data)
    running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = running_correct.float() / len(train_loader)
    running_loss_history.append(epoch_loss)
    running_correct_history.append(epoch_acc)

  print("===================================================")
  print("epoch: ", e + 1)
  print("training loss: {:.5f}, acc: {:5f}".format(epoch_loss, epoch_acc))
with torch.no_grad():
  acc_1=0
  acc_5=0
  for val_input, val_label in test_loader:
    
    val_input = val_input.to(device)
    val_label = val_label.to(device)
    val_outputs = model(val_input)

    _, val_preds_1 = torch.max(val_outputs, 1)
    _, five = torch.sort(val_outputs)
    five = five[-5:]
    test_correct_1 = torch.sum(val_preds_1 == val_label.data)
    if val_label.data in five:
      acc_5 += 1

    acc_1+=test_correct_1.float()
    
  print("TOP 1 acc: {:5f} %".format(acc_1*100/len(test_loader)))
  print("TOP 5 acc: {:5f} %".format(acc_5*100/len(test_loader)))

Files already downloaded and verified
Files already downloaded and verified
epoch:  1
training loss: 1.64867, acc: 40.818001
epoch:  2
training loss: 1.26521, acc: 55.262001
epoch:  3
training loss: 1.14008, acc: 60.384003
epoch:  4
training loss: 1.06882, acc: 62.950005
epoch:  5
training loss: 1.00753, acc: 65.248001
epoch:  6
training loss: 0.97265, acc: 66.210007
epoch:  7
training loss: 0.94581, acc: 67.522003
epoch:  8
training loss: 0.91604, acc: 68.424004
epoch:  9
training loss: 0.90482, acc: 68.990005
epoch:  10
training loss: 0.88755, acc: 69.336006
epoch:  11
training loss: 0.87288, acc: 70.080002
epoch:  12
training loss: 0.85438, acc: 70.676003
TOP 1 acc: 66.919998 %
TOP 5 acc: 100.000000 %
